In [1]:
# import
import pandas as pd
import random
from pathlib import Path
import re
import datetime

pd.options.display.max_columns = None

In [2]:
base_directories = [
    "/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ",
    "/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดอุตุนิยมวิทยา",
]

exclude_dirs = ["TEMP BP"]

output_dir = "cleaned_data"
base_dirs = base_directories

In [3]:
from pathlib import Path

# Create output directory
output_path = Path(output_dir)
output_path.mkdir(exist_ok=True)

# Get all xlsx files
all_xlsx_files = []
for base_dir in base_dirs:
    if any(exclude_dir in base_dir for exclude_dir in exclude_dirs):
        continue

    dir_path = Path(base_dir)
    if not dir_path.exists():
        print(f"Warning: Directory {base_dir} does not exist")
        continue
    all_xlsx_files.extend(list(dir_path.rglob("*.xlsx")))

# Remove exclude files
all_xlsx_files = [
    xlsx_file
    for xlsx_file in all_xlsx_files
    if not any(
        exclude_dir in str(xlsx_file) for exclude_dir in exclude_dirs
    )  # FIXED HERE
]

all_xlsx_files

[PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Banhuafai.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/BanHuayking.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Banmaechang.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Banmairattanakosin.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Bansadej.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Bansopmoe.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Bansoppad.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Banthasii.xlsx'),
 PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อน

In [4]:
def convert_thai_date_to_datetime(thai_date):
    """Convert Thai Buddhist Era (BE) dates to Gregorian (AD) dates."""
    if not isinstance(thai_date, str) or not thai_date.strip():
        return thai_date

    month_mapping = {
        "มกราคม": "January",
        "กุมภาพันธ์": "February",
        "มีนาคม": "March",
        "เมษายน": "April",
        "พฤษภาคม": "May",
        "มิถุนายน": "June",
        "กรกฏาคม": "July",
        "สิงหาคม": "August",
        "กันยายน": "September",
        "ตุลาคม": "October",
        "พฤศจิกายน": "November",
        "ธันวาคม": "December",
    }

    try:
        if not isinstance(thai_date, str) or not thai_date.strip():
            return thai_date  # Return NaT for empty/invalid values

        spx = thai_date.strip().split()
        if len(spx) != 3:
            return pd.NaT  # Ignore invalid formats

        day, thai_month, thai_year = spx
        month = month_mapping.get(thai_month)

        if not month:  # If the Thai month is unknown
            print(f"⚠️ Unknown month: {thai_month}")
            return pd.NaT

        # Convert Thai Buddhist Era year to Gregorian year
        gregorian_year = int(thai_year) - 543

        # Convert to datetime object
        date_string = f"{day} {month} {gregorian_year}"
        return datetime.datetime.strptime(date_string, "%d %B %Y")

    except Exception as e:
        print(f"⚠️ Error converting Thai date: {thai_date}, Error: {e}")
        return pd.NaT  # Return NaT for errors

In [5]:
# For all sheets
# import ipython terminate
import IPython.core.ultratb as ultratb

file_stop_error_file = []

all_file_dfs = []

for file_path in all_xlsx_files:
    xls = pd.ExcelFile(file_path)
    sheet_nums = len(xls.sheet_names)
    sheet_names = xls.sheet_names

    # file name without extension
    file_name = file_path.stem
    parent_dir = (
        file_path.parent.name
        if len(file_path.parent.name) > 5
        else str(file_path.parent.name) + "_" + str(file_path.parent.parent.name)
    )
    file_name = parent_dir + "#" + file_name

    print("\n\n\n")
    print("#" * 50)
    print(parent_dir)
    print(file_name, file_path)
    print("#" * 50)

    # Read all sheets
    all_dfs = []

    if sheet_nums > 1:
        raise ValueError("More than 1 sheet in the file")

    dfs = []
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        # check first 20 rows each sheet must have  less than 10 columns if not rise error

        is_no_data = False
        for ri, row in enumerate(df.itertuples(index=False)):
            if ri >= 20:  # Only check the first 20 rows
                break

            row_values = [col for col in row]  # Convert row to list

            if len(row_values) < 10:  # If 10 or more columns exist, log the error
                print(
                    f"⚠️ More than 10 columns found in row {ri}: {len(row_values)} columns"
                )
                file_stop_error_file.append(["<10 columns", file_path])
                is_no_data = True
                break

        if is_no_data:
            continue

        header_idx = -1
        for ri, row in enumerate(df.itertuples(index=False)):
            row_values = [
                str(col).lower().strip() if pd.notna(col) else "" for col in row
            ]
            print(row_values)

            # Detect "Date/Time" in first cell
            if "date/time" in row_values[0].lower():
                # Check if second cell onwards are mostly empty indicating fake header
                if all(cell == "" for cell in row_values[1:]):
                    header_idx = ri + 1  # real header is next row
                    print(
                        f"Fake header detected at row {ri}, real header at row {header_idx}"
                    )
                else:
                    header_idx = ri  # real header is current row
                    print(f"Real header detected at row {header_idx}")
                break

        if header_idx != -1:
            df.columns = df.iloc[header_idx]
            df = df.iloc[header_idx + 1 :].reset_index(drop=True)

            # Normalize columns clearly
            df.columns = ["Date/Time"] + [str(col).strip() for col in df.columns[1:]]

            # **Fix:** Ensure DataFrame has rows before checking the first row
            if not df.empty and str(df.iloc[0, 0]).strip().lower() == "date/time":
                print("Duplicate header detected in first data row, removing it.")
                df = df.iloc[1:].reset_index(drop=True)
        else:
            print("Could not find a valid header.")

        # Continue with further processing or footer removal logic...

        # print(df.head(10).to_markdown())

        # -------- End of find header ------------
        # df = find_footer(df)
        # ------ Find footer -------------
        dataframe = df.copy()
        row_index = None
        # check empty dataframe

        # find "date/time" in first column
        has_date_time = "date/time" in [
            str(col_name).lower().strip() for col_name in dataframe.columns
        ]

        if not has_date_time:
            print("No 'Date/Time' column found", file_path)
            continue

        for i, row in dataframe.iterrows():
            if "Station Down (D)=" in str(row["Date/Time"]):
                row_index = i
                break

        print("Remove footer at row index:", row_index)

        # Remove rows from the bottom up to and including the found row
        if row_index is not None:
            print("Removing rows from index:", row_index, -1)
            dataframe = dataframe.iloc[:row_index]

            # CLean NaN last 10 rows
            df_no10 = dataframe.iloc[:-10]
            df_cleaned = dataframe.tail(10).dropna(subset=["Date/Time"], how="any")
            dataframe = pd.concat([df_no10, df_cleaned], ignore_index=True)

        else:
            print("Text 'Station Down (D)=' not found")

        # Reset the index if needed
        dataframe = dataframe.reset_index(drop=True)
        df = dataframe
        # -------- End of find footer ------------
        dfs.append(df)

        # ----------- Series of print ------------
        # covrt date
        # check date/time is datetime or object if object maybe thai date convert to datetime
        print(df["Date/Time"].dtype)

        df["Date/Time"] = df["Date/Time"].apply(convert_thai_date_to_datetime)

        # ----------- SelectColumns ------------

        start_time = "1990-01-01 01:00:00"
        end_time = "1990-01-02 00:59:59"
        time_range = pd.date_range(start=start_time, end=end_time, freq="h")
        time_columns = [time.strftime("%H:%M:%S") for time in time_range.time]

        df = df.iloc[:, :25]
        df.columns = ["Date/Time"] + time_columns

        # ----------- RenameColumns ------------
        # rename Date/Time to Datetime

        # ----------- To Serialize ------------

        # Melt the DataFrame to transform columns into rows
        df_melted = df.melt(id_vars=["Date/Time"], var_name="Hour", value_name="Value")

        # Convert "Date/Time" to datetime
        df_melted["Date/Time"] = pd.to_datetime(df_melted["Date/Time"])

        # Combine Date/Time with Hour
        df_melted["Full_Timestamp"] = (
            df_melted["Date/Time"].astype(str) + " " + df_melted["Hour"]
        )

        # Convert to actual datetime format
        df_melted["Full_Timestamp"] = pd.to_datetime(
            df_melted["Full_Timestamp"], errors="coerce"
        )

        # Identify "00:00:00" and shift it to the next day
        mask = df_melted["Hour"] == "00:00:00"
        df_melted.loc[mask, "Full_Timestamp"] += pd.Timedelta(days=1)

        # Drop unnecessary columns and rename
        df_melted = df_melted[["Full_Timestamp", "Value"]].rename(
            columns={"Full_Timestamp": "Timestamp"}
        )

        df_melted_sorted = df_melted.sort_values(by="Timestamp").reset_index(drop=True)
        df = df_melted_sorted.copy()

        # df = df_melted

        # new Dataframe

        # # Melt the dataframe
        # melted_df = pd.melt(
        #     df, id_vars=["Date/Time"], var_name="Time", value_name="Value"
        # )

        # # df = df[["Datetime"] + [col for col in df.columns if col != "Datetime"]]

        # # melted_df = pd.melt(
        # #     df, id_vars=["Date/Time"], var_name="Time", value_name="Value"
        # # )

        # melted_df.rename(columns={"Date/Time": "Datetime"}, inplace=True)

        # melted_df["_time_dt"] = pd.to_datetime(
        #     melted_df["Time"], format="%H:%M:%S", errors="coerce"
        # ).dt.time

        # melted_df["Datetime"] += pd.to_timedelta(melted_df["_time_dt"].astype(str))
        # melted_df.loc[melted_df["Time"] == "00:00:00", "Datetime"] += pd.DateOffset(
        #     days=1
        # )

        # df = melted_df.drop(columns=["Time", "_time_dt"])

        # ----------- End of Serialize ------------

        print(df.head(10).to_markdown())
        print(df.tail(10).to_markdown())
        print(df_melted_sorted.head(10).to_markdown())
        print(df_melted_sorted.tail(10).to_markdown())
        print("-----------------------------------\n\n")

        # -------------- Save to CSV --------------
        # Save to CSV
        output_file = output_path / f"{file_name}.csv"
        df.to_csv(output_file, index=False)
        print(f"Saved to {output_file}")





##################################################
NO2 2009-2019
NO2 2009-2019#Banhuafai /mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/NO2 2009-2019/Banhuafai.xlsx
##################################################
['no2 station no.4 บ้านหัวฝาย', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['fatal error: maximum execution time of 30 seconds exceeded in d:\\www\\lib_cwyp\\lib.php on line 527', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['date/time', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00', '24:00', '24 avg', '1 hr>320', 'max']
Real header detected at row 2
Remove footer at row index: None
Text 'Station Down (D)=' not found
object
|    | Timestamp           |   Value 

In [6]:
file_stop_error_file

# remove duplicate rows

error_file = []
for file in file_stop_error_file:
    if file not in error_file:
        error_file.append(file)

error_file

[['<10 columns',
  PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/PM10 2009-2019/BanHuayking.xlsx')],
 ['<10 columns',
  PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/PM10 2009-2019/Banmairattanakosin.xlsx')],
 ['<10 columns',
  PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/PM10 2009-2019/Bansadej.xlsx')],
 ['<10 columns',
  PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/PM10 2009-2019/Banthasii.xlsx')],
 ['<10 columns',
  PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/PM10 2009-2019/Pratupha.xlsx')],
 ['<10 columns',
  PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/SO2 2009-2020/SO2 2010-2020/สถานีตรวจอากาศหลัก.xlsx')],
 ['<10 columns',
  PosixPath('/mnt/e/MayThesis2025/datasets/ข้อมูลย้อนหลัง10ปี/ผลการตรวจวัดคุณภาพอากาศ/TSP 2009-2019/Banthasii.xlsx')]]

In [7]:
all_file_dataframes

NameError: name 'all_file_dataframes' is not defined